# Pre-process the New Case Reports

In [ ]:
import pandas as pd
def prepro(filename):
    data = pd.read_excel(filename)
    contents=list(data['Original_Text_CN'])
    print('Have '+str(len(contents))+' case reports in total!')
    with open("datasets/raw/NER/test.txt","w",encoding='utf-8') as f:
        for content in contents:
            if isinstance(content,str):
                content=content.replace('\n','')
                for char in content:
                    f.write(char+'\t'+'O'+'\n')       
            else:
                print("ERROR FILE！")
                f.write('。'+'\n')
            f.write('\n')

if __name__ == "__main__":
    file_path = "NewCases.xlsx"
    prepro(file_path)

# Evaluation Script for Extracting New Entities

Python train_base_model_bert.py --train False

# Post_process the Entities in BIO to Readable Entities

In [ ]:
import pandas as pd
import re
import os

def get_gender(contents):
    pat1="男|女"
    pat2="\d{1,3}岁"

    contents=list(contents)
    xings=[]
    years=[]
    for content in contents:
        if isinstance(content,str):
            xing=re.findall(pat1,content)
            year=re.findall(pat2,content)
            if xing:
                xings.append(xing[0])
            else:
                xings.append("NA")
            if year:
                years.append(year[0])
            else:
                years.append("NA")   
        else:
            xings.append("NA")
            years.append("NA") 
    return xings,years

def time_deal(result,contents):
    colnames=['Arrival_Data','Date_Quarantine','Date_Symptom_Onset','Date_Hospitalisation','Date_Confirmation']
    pat='(\d{1,2})月'
    new=[] 
    for colname in colnames:
        items=list(result[colname])  
        newitems=[]
        for times in items:
            if isinstance(times,str):
                timelist=times.split(',')
                newtimelist=[]
                for time in timelist:
                    index=time.find('日')
                    newtime=time 
                    if index!=-1:
                        newtime=time[:index+1]                                   
                    if '年' not in newtime and  ('月'in newtime or '日' in newtime):
                        newtime='2020年'+newtime
                    newtimelist.append(newtime)
                newtimes=','.join(newtimelist)
            else:
                newtimes=times
            newitems.append(newtimes)
        result[colname]=pd.Series(newitems)
    # print(result)       

    pat1='(\d{2,4})?年(\d{1,2})月(\d{1,2})日'
    pat2='(\d{2,4})?年(\d{1,2})月'
    pat3='(\d{2,4})?年(\d{1,2})日'
    new=[]
    for colname in colnames:
        items=list(result[colname]) 
        newitems=[]
        for times in items:
            if isinstance(times,str):
                timelist=times.split(',')
                newtimelist=[]
                for time in timelist:                
                    t1=re.findall(pat1,time)
                    if t1:
                        t=t1[0]               
                        nt=[]
                        if t[0]=='':
                            nt.append('2020')
                        else:
                            nt.append(t[0])
                        if t[1]!='':
                            nt.append(t[1])
                        else:
                            nt.append(' ')
                        if t[2]!='':
                            nt.append(t[2])
                        else:
                            nt.append(' ')
                        nti='/'.join(nt)

                        newtimelist.append(nti)                                                
                    else:
                        t2=re.findall(pat2,time)
                        if t2:
                            t=t2[0]               
                            nt=[]
                            if t[0]=='':
                                nt.append('2020')
                            else:
                                nt.append(t[0])
                            if t[1]!='':
                                nt.append(t[1])
                            else:
                                nt.append(' ')
                            nt.append(' ')
                            nti='/'.join(nt)

                            newtimelist.append(nti)
                            
                        else:
                            t3=re.findall(pat3,time)
                            if t3:
                                t=t3[0]               
                                nt=[]
                                if t[0]=='':
                                    nt.append('2020')
                                else:
                                    nt.append(t[0])
                                nt.append(' ')
                                
                                if t[1]!='':
                                    nt.append(t[1])
                                else:
                                    nt.append(' ')
                                nti='/'.join(nt)

                                newtimelist.append(nti)
                            elif '/' in time:
                                newtimelist.append(time)
                newtimes=','.join(newtimelist)
            else:
                newtimes=times
            newitems.append(newtimes)            
        result[colname]=pd.Series(newitems)   
    
    for colname in colnames:
        times=list(result[colname]) 
        newitems=[]
    #     for i in range(len(times)):
        for i in range(len(times)):
            content=contents[i]
            if isinstance(times[i],str):
                timelist=times[i].split(',')
                newtimelist=[]
                for time in timelist:
                    date=time.split('/')
                    if len(date)<3:
                        # print(i)
                        continue
                    if date[2] != ' ':##过滤没有日的日期                    
                        newtimelist.append(time)
                newtimelist=list(set(newtimelist))
                newnewtimelist=[]
                for time in newtimelist:##补充月份，找该日期最近提及的月份
                    date=time.split('/')
                    if date[1]==' ':
                        index=content.find(date[2]+'日')
                        months=re.findall(pat,content[:index])
                        if months:
                            newnewtimelist.append(date[0]+'/'+months[-1]+'/'+date[2])
                        else:
                            newnewtimelist.append(date[2])
                    else:
                        newnewtimelist.append(time)
                newtimes=','.join(newnewtimelist)
            else:
                newtimes=times[i]
            newitems.append(newtimes)
        result[colname]=pd.Series(newitems)
        

    return result

def afterpro(filename):
    labels=['QYL','ZZL','MDL','MDT','GLT','FBT','JZT','ZSI','QZT']
    titles=['Place_of_Departure','Place_of_Transit','Place_of_Destination','Arrival_Data','Date_Quarantine','Date_Symptom_Onset','Date_Hospitalisation','Place_Hospitalisation','Date_Confirmation',]
    with open(filename,"r",encoding='utf-8') as f:
        data=f.read()
    contents=data.split('\n\n')

    all=[]
    for content in contents:
        lines=content.split('\n')
        newlines=[]
        for line in lines:
            chars=line.split()
            newlines.append(chars)
        all.append(newlines)           

    ress=[]
    for chars in all[:-1]:
        res=[]
        items={}
        i=0
        while i<len(chars):
            if 'B' in chars[i][2]:
                key=chars[i][2][2:]
                value=chars[i][0]
                i+=1
                while 'I' in chars[i][2]:
                    value+=chars[i][0]
                    i+=1
                if key in items :
                    items[key]=items[key]+','+value
                else:
                    items[key]=value
            else:
                i+=1

        for key in labels:
            if key in items:
                res.append(items[key])
            else:
                res.append("NA")
        ress.append(res)

    data = pd.read_excel('NewCases.xlsx')
    contents_=list(data['Original_Text_CN'])
        
    result=pd.DataFrame(ress,columns=titles)
    
    
    result=time_deal(result,contents_)
    cont=data['Original_Text_CN'].values  
    result.insert(0,'Original_Text_CN',cont)
    xings,years=get_gender(cont)
    result.insert(loc=10,column='Gender',value=xings)
    result.insert(loc=11,column='Age',value=years)

    result.to_excel('result.xlsx',encoding='utf-8')


if __name__ == "__main__":
    afterpro('ckpt/wzz/base_model_wzz1_1_Roberta_wwm_wzz_1_0918/test_result.txt')